In [1]:
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
from keras.preprocessing import image
from math import *

import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
tf.python.control_flow_ops = control_flow_ops

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, InputLayer
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.applications.vgg16 import VGG16

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

%matplotlib inline

Using TensorFlow backend.


In [2]:
data_path = './driving-data/'
img_rows, img_cols = 160, 320
crop_top = 0

In [53]:
def load_data(normalizer=255.0, vgg=False):
    columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

    print('Loading driving log ...')

    driving_log = pd.read_csv(data_path+'driving_log.csv', names=columns)
    num_rows = len(driving_log.index)

    if vgg:
        train_images = np.zeros((num_rows*3, 224, 224, 3))
    else:
        train_images = np.zeros((num_rows*3, img_rows, img_cols, 3))
    train_steering = np.zeros(num_rows*3)

    for index, row in tqdm_notebook(driving_log.iterrows(), unit=' rows', total=num_rows):
        fname  = os.path.basename(row['center'])
        fname1 = os.path.basename(row['left'])
        fname2 = os.path.basename(row['right'])
        # Normalized YUV
        if vgg:
            img = image.load_img(data_path+'IMG/'+fname, target_size=(224, 224))
            img = image.img_to_array(img).astype(np.float32)
            
            img1 = image.load_img(data_path+'IMG/'+fname1, target_size=(224, 224))
            img1 = image.img_to_array(img1).astype(np.float32)
            
            img2 = image.load_img(data_path+'IMG/'+fname2, target_size=(224, 224))
            img2 = image.img_to_array(img2).astype(np.float32)
        else:    
            img  = imread(data_path+'IMG/'+fname, False, 'YCbCr').astype(np.float32)/normalizer
            img1 = imread(data_path+'IMG/'+fname1, False, 'YCbCr').astype(np.float32)/normalizer
            img2 = imread(data_path+'IMG/'+fname2, False, 'YCbCr').astype(np.float32)/normalizer

        train_images[3*index] = img
        train_images[3*index+1] = img1
        train_images[3*index+2] = img2
        
        train_steering[3*index] = np.float32(row['steering'])
        train_steering[3*index+1] = np.float32(row['steering'])+2./25.
        train_steering[3*index+2] = np.float32(row['steering'])-2./25.
        

    print('Loaded', num_rows, 'rows.')
    return train_images, train_steering

def image_generator(driving_log, normalizer=255.0, vgg=False, steering_shift=3., steering_max=25.):
    
    for index, row in driving_log.iterrows():
        fname  = os.path.basename(row['center'])
        fname1 = os.path.basename(row['left'])
        fname2 = os.path.basename(row['right'])
        
        if vgg:
            img = image.load_img(data_path+'IMG/'+fname, target_size=(224, 224))
            img = image.img_to_array(img).astype(np.float32)
            
            img1 = image.load_img(data_path+'IMG/'+fname1, target_size=(224, 224))
            img1 = image.img_to_array(img1).astype(np.float32)
            
            img2 = image.load_img(data_path+'IMG/'+fname2, target_size=(224, 224))
            img2 = image.img_to_array(img2).astype(np.float32)
        else:
            # Normalized YUV
            img  = imread(data_path+'IMG/'+fname, False, 'YCbCr').astype(np.float32)
            img1 = imread(data_path+'IMG/'+fname1, False, 'YCbCr').astype(np.float32)
            img2 = imread(data_path+'IMG/'+fname2, False, 'YCbCr').astype(np.float32)
        
        yield img, np.float32(row['steering'])
#         yield np.fliplr(img), -np.float32(row['steering'])
        
        yield img1, np.float32(row['steering'])+steering_shift/steering_max
#         yield np.fliplr(img1), -(np.float32(row['steering'])+steering_shift/steering_max)
        
        yield img2, np.float32(row['steering'])-steering_shift/steering_max
#         yield np.fliplr(img2), -(np.float32(row['steering'])-steering_shift/steering_max)
        
def normalize_color(image_data):
    a = -0.5
    b = +0.5
    
    Xmin = 0.0
    Xmax = 255.0

    norm_img = np.empty_like(image_data, dtype=np.float32)

    norm_img = a + (image_data - Xmin)*(b-a)/(Xmax - Xmin)
    return norm_img

def batch_generator(driving_log, batch_size=32, *args, vgg=False, **kwargs):
    num_rows = len(driving_log.index)
    train_images = np.zeros((batch_size, img_rows, img_cols, 3))
    train_steering = np.zeros(batch_size)
    ctr = None
    while 1:        
        for j in range(batch_size):
            # Reset generator if over bounds
            if ctr is None or ctr >= num_rows:
                ctr = 0
                images = image_generator(driving_log, *args, **kwargs)
            train_images[j], train_steering[j] = next(images)
            ctr += 1
        yield normalize_color(train_images), (train_steering+1.)/2.

In [54]:
columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
driving_log = pd.read_csv(data_path+'driving_log.csv', names=columns)
num_rows = len(driving_log.index)
train_data = batch_generator(driving_log.iloc[:-1000], batch_size=50)
val_data = batch_generator(driving_log.iloc[-1000:], batch_size=50)

In [47]:
# model_name = 'model_1'
# with open(model_name+'.json', 'r') as jfile:
#     # model = model_from_json(json.load(jfile))
#     model = model_from_json(jfile.read())

# model.compile("adam", "mse")
# weights_file = model_name+'.h5'
# model.load_weights(weights_file)
# model.summary()
# print(X_train.shape)

In [55]:
model = Sequential()
model.add(Convolution2D(24, 5, 5, border_mode='same',
                        input_shape=(img_rows, img_cols, 3), subsample=(2,2)))
# model.add(BatchNormalization())
model.add(Activation('relu', name='Conv1'))

model.add(Convolution2D(36, 5, 5, subsample=(2,2)))
# model.add(BatchNormalization())
model.add(Activation('relu', name='Conv2'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))

model.add(Convolution2D(48, 5, 5))#, subsample=(2,2)))
model.add(BatchNormalization())
model.add(Activation('relu', name='Conv3'))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu', name='Conv4'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
# model.add(Convolution2D(64, 3, 3))
# model.add(BatchNormalization())
# model.add(Activation('relu', name='Conv5'))
# model.add(MaxPooling2D((2,2)))
# model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1164))
# model.add(BatchNormalization())
model.add(Activation('relu', name='FC1'))
model.add(Dense(100))
# model.add(BatchNormalization())
model.add(Activation('relu', name='FC2'))
model.add(Dense(50))
# model.add(BatchNormalization())
model.add(Activation('relu', name='FC3'))
model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('relu', name='FC4'))
model.add(Dense(1, activation='relu', name='output'))

In [57]:
opt = Adam(lr=1e-4)
model.compile(optimizer='adam', loss='mse')

In [60]:
# h = model.fit(X_train, y_train, batch_size=32, nb_epoch=5, verbose=1)
h = model.fit_generator(train_data, validation_data = val_data,
                        samples_per_epoch = 10000,
                        nb_val_samples = 1000,
                        nb_epoch=10, verbose=1)

Epoch 1/10
10000/10000 [==============================] - 63s - loss: 0.0024 - val_loss: 0.0332
Epoch 2/10
10000/10000 [==============================] - 60s - loss: 0.0022 - val_loss: 0.0154
Epoch 3/10
10000/10000 [==============================] - 59s - loss: 0.0019 - val_loss: 0.0133
Epoch 4/10
10000/10000 [==============================] - 59s - loss: 0.0020 - val_loss: 0.0069
Epoch 5/10
10000/10000 [==============================] - 59s - loss: 0.0019 - val_loss: 0.0096
Epoch 6/10
10000/10000 [==============================] - 59s - loss: 0.0016 - val_loss: 0.0098
Epoch 7/10
10000/10000 [==============================] - 59s - loss: 0.0016 - val_loss: 0.0050
Epoch 8/10
10000/10000 [==============================] - 59s - loss: 0.0016 - val_loss: 0.0128
Epoch 9/10
10000/10000 [==============================] - 58s - loss: 0.0016 - val_loss: 0.0355
Epoch 10/10
10000/10000 [==============================] - 59s - loss: 0.0012 - val_loss: 0.0051


In [51]:
model.evaluate_generator(val_data, 10)

0.0011621094308793545

In [61]:
model_json = model.to_json()
model_name = 'model_4b'
with open(model_name+'.json', "w") as json_file:
    json_file.write(model_json)

model.save_weights(model_name+'.h5')

In [ ]:
from keras.applications import VGG16
vgg = VGG16(weights='imagenet', include_top=False)

In [ ]:
vgg.summary()

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model2 = Model(input=vgg.input, output=vgg.get_layer('block3_pool').output)
    
img_path = 'driving-data/IMG/center_2016_11_27_23_18_18_813.jpg'
img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model2.predict(x)
print(block4_pool_features.shape)

# plt.figure()
# for i in range(16):
#     plt.subplot(4, 4, i+1)
#     plt.imshow(block4_pool_features[0,:,:,16+i])
    
# plt.show()

In [ ]:
model2.layers[0] = InputLayer(input_shape=(224,224,3), input_dtype=np.float32)

for l in model2.layers:
    l.trainable=False
    
model = Sequential(model2.layers)#, input_shape=(12, 40, 256))  # 20,40,256
model.add(Convolution2D(24, 5, 5))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(36, 5, 5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))

model.add(Convolution2D(48, 5, 5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(1154))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(50))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))
model.add(Dense(1))

In [ ]:
X1_train, y1_train = load_data(normalizer=1.0, vgg=True)
X_train, X_val, y_train, y_val = train_test_split(X1_train, y1_train, test_size=0.05, random_state=0xdeadbeef)
del X1_train, y1_train
X2_train = preprocess_input(X_train)
X2_val = preprocess_input(X_val)

In [ ]:
opt = Adam(lr=1e-3)
model.compile(optimizer=opt, loss='mae')
model.predict(x)

In [ ]:
h = model.fit(X2_train, y_train, batch_size=35, nb_epoch=10,
    validation_data=(X2_val, y_val), verbose=1)

In [ ]:
model_json = model.to_json()
model_name = 'vgg_01'
with open(model_name+'.json', "w") as json_file:
    json_file.write(model_json)

model.save_weights(model_name+'.h5')

In [ ]:
model.predict(X2_train[0:10,:])